In [1]:
import numpy as np
import time
import qcodes
import pyvisa as visa
# enable to get a more debuggable output
#visa.log_to_screen()

from qcodes.instrument_drivers.yokogawa.GS200 import GS200
py_rm = visa.ResourceManager('@py')
ni_rm = visa.ResourceManager()

print(py_rm)
print(ni_rm)

/home/nikolai/miniconda3/envs/qcodes/lib/python3.9/site-packages/gpib_ctypes/gpib/gpib.py:54: UserWarning: GPIB library not found. Please manually load it using _load_lib(filename). All GPIB functions will raise OSError until the library is manually loaded.
  warnings.warn(message)


Resource Manager of Visa Library at py
Resource Manager of Visa Library at /usr/lib/x86_64-linux-gnu/libvisa.so.21.5.0


In [2]:
# for some reason, the USB access has to be initialized with the python resource manager
# the resource is opened to be recognizable by the ni drivers
py_rm.open_resource(py_rm.list_resources()[1])

print("visa-py:" + str(py_rm.list_resources()))
print("visa-ni:" + str(ni_rm.list_resources()))

visa-py:('ASRL/dev/ttyS4::INSTR', 'USB0::2849::57::91LA25023::0::INSTR')
visa-ni:('USB0::0x0B21::0x0039::91LA25023::INSTR', 'ASRL5::INSTR')


In [3]:
# intialize the instrument with the following parameters
gs_addr = ni_rm.list_resources()[0]
gs = GS200("gs200", address=gs_addr, terminator="\n")

Connected to: YOKOGAWA GS210 (serial:91LA25023, firmware:1.05) in 0.26s


In [4]:
#gs.snapshot()

In [5]:
gs.source_mode()

'VOLT'

In [6]:
gs.reset()
gs.auto_range(True)

In [7]:
gs.source_mode('CURR')
gs.source_mode()

'CURR'

In [82]:
gs.output('off')
gs.output()

'off'

In [83]:
gs.output('on')
gs.current(0)
t1 = time.time()
gs.ramp_current(0.02, 0.001, 1)
t2 = time.time()
print("Ramping took {:.4} seconds".format(t2-t1))

Ramping took 19.05 seconds


In [72]:
gs.current(0)


In [73]:
gs.close()